## Cài đặt thư viện 

In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import math
from keras.models import Sequential
from keras.layers import Dense,LSTM
from keras.wrappers.scikit_learn import KerasRegressor
import os.path
from os import path
from pylab import rcParams
from random import randrange
from pandas import Series
from matplotlib import pyplot
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.tsatools import freq_to_period
from sklearn import model_selection, preprocessing, feature_selection, ensemble, linear_model, metrics, decomposition
import pickle
import os
import joblib

In [2]:
import tensorflow as tf 

## Xử lý + Khảo sát bộ dữ liệu 

In [3]:
# Đọc dữ liệu từ folder 
df = pd.read_csv("D:/Github/Electric-Power-Consumption/dataset/powerconsumption.csv", sep=",")

df.head(5)

,Datetime,Temperature,Humidity,WindSpeed,GeneralDiffuseFlows,DiffuseFlows,PowerConsumption_Zone1,PowerConsumption_Zone2,PowerConsumption_Zone3
0,1/1/2017 0:00,6.559,73.8,0.083,0.051,0.119,34055.69620,16128.87538,20240.96386
1,1/1/2017 0:10,6.414,74.5,0.083,0.070,0.085,29814.68354,19375.07599,20131.08434
2,1/1/2017 0:20,6.313,74.5,0.080,0.062,0.100,29128.10127,19006.68693,19668.43373
3,1/1/2017 0:30,6.121,75.0,0.083,0.091,0.096,28228.86076,18361.09422,18899.27711
4,1/1/2017 0:40,5.921,75.7,0.081,0.048,0.085,27335.69620,17872.34043,18442.40964


In [4]:
# chuyển cột Datetime thành Index 
df.set_index("Datetime", inplace = True) 

df.head(10)

,Temperature,Humidity,WindSpeed,GeneralDiffuseFlows,DiffuseFlows,PowerConsumption_Zone1,PowerConsumption_Zone2,PowerConsumption_Zone3
Datetime,,,,,,,,
1/1/2017 0:00,6.559,73.8,0.083,0.051,0.119,34055.69620,16128.87538,20240.96386
1/1/2017 0:10,6.414,74.5,0.083,0.070,0.085,29814.68354,19375.07599,20131.08434
1/1/2017 0:20,6.313,74.5,0.080,0.062,0.100,29128.10127,19006.68693,19668.43373
1/1/2017 0:30,6.121,75.0,0.083,0.091,0.096,28228.86076,18361.09422,18899.27711
1/1/2017 0:40,5.921,75.7,0.081,0.048,0.085,27335.69620,17872.34043,18442.40964
1/1/2017 0:50,5.853,76.9,0.081,0.059,0.108,26624.81013,17416.41337,18130.12048
1/1/2017 1:00,5.641,77.7,0.080,0.048,0.096,25998.98734,16993.31307,17945.06024
1/1/2017 1:10,5.496,78.2,0.085,0.055,0.093,25446.07595,16661.39818,17459.27711
1/1/2017 1:20,5.678,78.1,0.081,0.066,0.141,24777.72152,16227.35562,17025.54217


In [5]:
data_input = df[['Temperature','Humidity','WindSpeed','GeneralDiffuseFlows','DiffuseFlows']] 
data_label_1 = df[['PowerConsumption_Zone1']]
data_label_2 = df[['PowerConsumption_Zone2']]
data_label_3 = df[['PowerConsumption_Zone3']]

In [6]:
data_input = df[['Temperature','Humidity','WindSpeed','GeneralDiffuseFlows','DiffuseFlows']] 
data_label = df[['PowerConsumption_Zone1','PowerConsumption_Zone2','PowerConsumption_Zone3']]

In [7]:
data_input = tf.constant(data_input)
data_label_1 = tf.constant(data_label_1)
data_label_2 = tf.constant(data_label_2)
data_label_3 = tf.constant(data_label_3)
data_label = tf.constant(data_label)

In [8]:
data_input_output = tf.constant(df)

In [9]:
X_5_70p = []
Y_3_30p = []

n_future = 3 
n_past = 7 

for i in range(n_past, len(data_input_output) - n_future + 1):
  X_5_70p.append(data_input_output[i - n_past:i, 0:8])
  Y_3_30p.append(data_input_output[i : i + n_future, 5:8])

X_5_70p, Y_3_30p = tf.constant(np.array(X_5_70p)), tf.constant(np.array(Y_3_30p))

# y_test_3_3days = y_test_3_3days.reshape(len(y_test_3_3days),n_future)
# y_train_3_3days = y_train_3_3days.reshape(len(y_train_3_3days),n_future)

In [10]:
X_5_70p = []
Y_3_30p = []

n_future = 3 
n_past = 7 

for i in range(n_past, len(data_input_output) - n_future + 1):
  X_5_70p.append(data_input_output[i - n_past:i, 0:8])
  Y_3_30p.append(data_input_output[i : i + n_future, 5:8])

X_5_70p, Y_3_30p = tf.Variable(X_5_70p), tf.Variable(Y_3_30p)

In [11]:
size_train = int(len(df)*0.70)
X_5_70p_train = X_5_70p[:size_train] 
X_5_70p_test = X_5_70p[size_train:]
Y_3_30p_train = Y_3_30p[:size_train] 
Y_3_30p_test = Y_3_30p[size_train:]

# y_test_3_3days = y_test_3_3days.reshape(len(y_test_3_3days),n_future)
# y_train_3_3days = y_train_3_3days.reshape(len(y_train_3_3days),n_future)

In [12]:
from keras.layers import Dropout

In [ ]:
model_LSTM_1_3days = Sequential()
model_LSTM_1_3days.add(LSTM(64, activation='relu', input_shape=(X_5_70p_train.shape[1], X_5_70p_train.shape[2]), return_sequences=True))
model_LSTM_1_3days.add(LSTM(32, activation='relu', return_sequences=False))
model_LSTM_1_3days.add(Dropout(0.02))
model_LSTM_1_3days.add(Dense(25))
model_LSTM_1_3days.add(Dense(Y_3_30p_train.shape[1]))
model_LSTM_1_3days.compile(optimizer = 'adam', loss = 'mean_squared_error')
model_LSTM_1_3days.fit(X_5_70p_train,Y_3_30p_train, epochs=1, batch_size=4, validation_split=0.1, verbose=1)
# Dự đoán trên tập test bằng mô hình LSTM 
y_pred_test_LSTM_1_3days = model_LSTM_1_3days.predict(X_5_70p_test)